In [184]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import gensim.downloader as gensim
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
import re

In [12]:
data = pd.read_csv('banks.csv', sep='\t')
data = data.drop('idx', axis =1)
data['Score'] = data['Score'].replace({'Negative': 0, 'Positive': 1})

In [13]:
model = gensim.load("word2vec-ruscorpora-300")

In [14]:
X,y = data['Score'].values, data['Text'].values
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.33, random_state=69)
print(f'shape of train data is {x_train.shape}')
print(f'shape of test data is {x_test.shape}')

shape of train data is (9379,)
shape of test data is (4620,)


In [27]:
y_train

array(['Вся предыстория данного отзыва описана здесь http://www.banki.ru/services/responses/bank/?responseID=5290660&code=9f919b2d9 . Если кратко, то 2 года назад Альфа-Банк мошенническим путем выдал потребительский кредит неустановленному лицу по ксерокопии моего паспорта и пытается взыскать данную задолженность с меня.  В результате моего судебного иска к банку суд постановил признать договор недействительным в декабре 2012. Дальнейшие действия банка детально описаны в предыдущем отзыве. После моих многократных обращений в банк и отставленного отзыва банк (исх N 077.2/65056 от 14.08.13) проинформировал об исполнении Банком решение суда. НО! Данная информация является ложной, т.к. по прошествии 3 месяцев после утверждения банка счет на мое имя не закрыт, и я числюсь должником банка. Об этом свидетельствует выписка со счета, выданная мне в отделении Альфа-Банка ДО Людмила 06.11.13. Я также числюсь злостным должником в БКИ, согласно выписке от 30.10.13. Т.е. Альфа-Банк предоставляет зав

In [119]:
tokenizer = RegexpTokenizer("[\w']+")

def preprocess(data):
    stop = set(stopwords.words('russian'))
    y_train = []
    for text in data:
        text = re.sub(r'http\S+', '', text)
        sentence_tokens = [i for i in tokenizer.tokenize(text.lower()) if i not in stop]
        lemmatized = []
        for word in sentence_tokens:
            lemma = morph.parse(word)[0].normal_form
            lemma = lemma + '_'
            lemma = lemma + str(morph.parse(word)[0][1])[0:4]
            lemmatized.append(lemma)
        y_train.append(lemmatized)

    return y_train

In [149]:
testing_data = preprocess(y_train) # takes 4.5 minutes

In [160]:
def get_vector(x):
    '''Input: Single token''' 
    try:
        return model.get_vector(x)
    except:
        return np.zeros(shape=300) #If the word is out of vocab, then return a 300 dim vector filled with zeros
def vector_sum(x):
    '''Input:List of word vectors'''
    return np.sum(x,axis=0)
def text_to_vector(lists_of_tokens):
    review_vectors = []
    for reviews in lists_of_tokens:
        vectors = []
        for token in reviews:
            vectors.append(get_vector(token))
        review_vectors.append(vector_sum(vectors))
    return review_vectors

In [156]:
review_vectors = text_to_vector(testing_data) #basically our y_train with review vectors

In [165]:
type(x_train)

numpy.ndarray

## Creating a Logistic Regression Model

In [166]:
ml_model = LogisticRegression(solver='liblinear', random_state=69)
ml_model.fit(review_vectors, x_train)

LogisticRegression(random_state=69, solver='liblinear')

In [171]:
p_pred = ml_model.predict_proba(review_vectors)
y_pred = ml_model.predict(review_vectors)
score_ = ml_model.score(review_vectors, x_train)
conf_m = confusion_matrix(x_train, y_pred)
report = classification_report(x_train, y_pred)

In [177]:
print(report)

              precision    recall  f1-score   support

           0       0.89      0.88      0.89      4694
           1       0.88      0.89      0.89      4685

    accuracy                           0.89      9379
   macro avg       0.89      0.89      0.89      9379
weighted avg       0.89      0.89      0.89      9379



In [179]:
y_test = preprocess(y_test)

In [182]:
y_test = text_to_vector(y_test)

In [183]:
#checking on test set
y_pred = ml_model.predict(y_test)
accuracy = accuracy_score(x_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8718614718614719


##### Logistic Regression достаточно точная, а также хорошо подходит для задачи бинарной классификации